## Идея заключается в том, чтобы в качестве свертки использовать не случайно инициализированную матрицу,  а матрицу посчитанную по всей трейн выборке с использованием PCA


Статья: **Image noise types recognition using CNN with PCA**

![title](img/pcanet.png)

0. все изображения привести к виду (m, m) 

1. Пусть у нас N изображений каждая (m, m). Вытаскиваем из одного изображения патчи размера $(k_a, k_a)$ вокруг каждого пикселя со stride = 1 на $a$ той итерации. 

2. Центрируем патчи: для каждого патча вычисляем среднее и вычитаем из каждого элемента патча среднее значение, для i-того изображения:
$\overline{X_i} = [\overline{x_1}, ..., \overline{x_N}] \subset \mathbb{R}^{k^2 x m^2}$

3. Проделываем это для N обучающих изображений: $X = [\overline{X_1}, ..., \overline{X_N}] \subset \mathbb{R}^{k^2 x Nm^2}$

4. для $ XX^T$ находим L **первых собственных векторов**, каждый из которых имеет размерность $ \mathbb{R}^{k^2}$, после этого делаем reshape до $ \mathbb{R}^{k x k}$ и получаем L искомых фильтров

Важно учесть, что первый фильтр работает с изображениями напрямую -> его можно просчитать заранее, в то время как остальные фильтры готовятся после предыдущего слоя!!!

In [6]:
test_image = np.arange(20).reshape(4,5)
test_image

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19]])

In [7]:
np.pad(test_image, (3, 3), 'constant', constant_values=0)

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  1,  2,  3,  4,  0,  0,  0],
       [ 0,  0,  0,  5,  6,  7,  8,  9,  0,  0,  0],
       [ 0,  0,  0, 10, 11, 12, 13, 14,  0,  0,  0],
       [ 0,  0,  0, 15, 16, 17, 18, 19,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

In [42]:
# Step 1: get patches k x k, где k - НЕЧЕТНЫЕ, чтобы можно было приложить к центру изображения
# TO DO: написать тесты

def get_patches(k, image) -> np.ndarray:
    padded_image = np.pad(image, (k // 2, k // 2), 'constant', constant_values=0) # можно не 0
    
    print(padded_image)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            patch = padded_image[i:i+k, j:j+k]
            print(patch)
            print()
            
get_patches(3, test_image)

[[ 0  0  0  0  0  0  0]
 [ 0  0  1  2  3  4  0]
 [ 0  5  6  7  8  9  0]
 [ 0 10 11 12 13 14  0]
 [ 0 15 16 17 18 19  0]
 [ 0  0  0  0  0  0  0]]
[[0 0 0]
 [0 0 1]
 [0 5 6]]

[[0 0 0]
 [0 1 2]
 [5 6 7]]

[[0 0 0]
 [1 2 3]
 [6 7 8]]

[[0 0 0]
 [2 3 4]
 [7 8 9]]

[[0 0 0]
 [3 4 0]
 [8 9 0]]

[[ 0  0  1]
 [ 0  5  6]
 [ 0 10 11]]

[[ 0  1  2]
 [ 5  6  7]
 [10 11 12]]

[[ 1  2  3]
 [ 6  7  8]
 [11 12 13]]

[[ 2  3  4]
 [ 7  8  9]
 [12 13 14]]

[[ 3  4  0]
 [ 8  9  0]
 [13 14  0]]

[[ 0  5  6]
 [ 0 10 11]
 [ 0 15 16]]

[[ 5  6  7]
 [10 11 12]
 [15 16 17]]

[[ 6  7  8]
 [11 12 13]
 [16 17 18]]

[[ 7  8  9]
 [12 13 14]
 [17 18 19]]

[[ 8  9  0]
 [13 14  0]
 [18 19  0]]

[[ 0 10 11]
 [ 0 15 16]
 [ 0  0  0]]

[[10 11 12]
 [15 16 17]
 [ 0  0  0]]

[[11 12 13]
 [16 17 18]
 [ 0  0  0]]

[[12 13 14]
 [17 18 19]
 [ 0  0  0]]

[[13 14  0]
 [18 19  0]
 [ 0  0  0]]



In [132]:
def get_centered_patches(k, image) -> np.ndarray:
    padded_image = np.pad(image, (k // 2, k // 2), 'constant', constant_values=0) # можно не 0
    X_im = []
    # print(padded_image)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            patch_matrix = padded_image[i:i+k, j:j+k]
            patch_vector = (patch_matrix.flatten()).astype(np.float32)
            #mean = np.mean(patch_vector)
            #patch_vector -= mean
            #print(patch_vector)
            X_im.append(patch_vector)
    return np.array(X_im).T

            
get_centered_patches(3, test_image).shape

(9, 20)

In [ ]:
# Step 3: проделать это с N изображениями, и собрать единую ДВУМЕРНУЮ матрицу


In [52]:
# numpy concatenate вроде как самый быстрый вариант, так как нужно будет это сделать для N изображений, где N - большое число

a = np.array([[1, 2], [3, 4]])
b = np.array([[5, 6], [7, 8]])
print(a)
print()
print(b)
np.concatenate((a,b), axis=1)

[[1 2]
 [3 4]]

[[5 6]
 [7 8]]


array([[1, 2, 5, 6],
       [3, 4, 7, 8]])

In [77]:
%%time

import cProfile

def compute_n_images():
    k = 3
    N = 600
    m = 64

    np.random.seed(0)

    all_patches = np.zeros((k ** 2, N * m ** 2))
    for i in range(N):
        #image_arr = Image.open()
        #gray = to_gray(image_arr)
        gray_im = np.random.rand(m, m)
        image_patches = get_centered_patches_profiled(k, gray_im) # k ** 2 x m ** 2
        #print(image_patches)
        # print(image_patches.shape, m**2)
        all_patches[:, i:i+m**2] = image_patches

    print(all_patches.shape)
    #print(all_patches)
    #del all_patches

cProfile.run('compute_n_images()')

(9, 2457600)
         41807441 function calls (41806241 primitive calls) in 54.812 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      600    0.001    0.000    0.005    0.000 <__array_function__ internals>:2(around)
  2457600    1.591    0.000   35.796    0.000 <__array_function__ internals>:2(mean)
      600    0.001    0.000    0.056    0.000 <__array_function__ internals>:2(pad)
      600    0.001    0.000    0.007    0.000 <__array_function__ internals>:2(round_)
      600   12.196    0.020   54.199    0.090 <ipython-input-72-9cbab2703260>:1(get_centered_patches)
        1    0.000    0.000   54.812   54.812 <string>:1(<module>)
        1    0.584    0.584   54.812   54.812 <timed exec>:3(compute_n_images)
     1200    0.001    0.000    0.004    0.000 _asarray.py:16(asarray)
  2457600    0.672    0.000    1.382    0.000 _asarray.py:88(asanyarray)
  2457600   16.787    0.000   29.157    0.000 _methods.py:134(_mean)
  2

In [137]:
def get_centered_patches_profiled(k, image) -> np.ndarray:
    padded_image = np.pad(image, (k // 2, k // 2), 'constant', constant_values=0) # можно не 0
    X_im = []
    #padded_image.astype(np.float32)
    # print(padded_image)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            patch_matrix = padded_image[i:i+k, j:j+k]
            X_im.append(patch_matrix)

    X_im = np.array(X_im, dtype=np.float32).T # k x k x m^2
    X_im = X_im.reshape(-1, X_im.shape[2])
    patch_mean = np.mean(X_im, axis=0)

    X_im -= patch_mean.reshape(-1, patch_mean.shape[0])
    return X_im

In [128]:
def matprint(mat, fmt="g"):
    col_maxes = [max([len(("{:"+fmt+"}").format(x)) for x in col]) for col in mat.T]
    for x in mat:
        for i, y in enumerate(x):
            print(("{:"+str(col_maxes[i])+fmt+"}").format(y), end="  ")
        print("")

In [131]:
np.set_printoptions(precision=2)
patches1 = get_centered_patches_profiled(3, test_image)
matprint(patches1)

0  0  0  0  0   0   0   1   2   3   0   5   6   7   8   0  10  11  12  13  
0  0  1  2  3   0   5   6   7   8   0  10  11  12  13   0  15  16  17  18  
0  5  6  7  8   0  10  11  12  13   0  15  16  17  18   0   0   0   0   0  
0  0  0  0  0   0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  
0  1  2  3  4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  
5  6  7  8  9  10  11  12  13  14  15  16  17  18  19   0   0   0   0   0  
0  0  0  0  0   1   2   3   4   0   6   7   8   9   0  11  12  13  14   0  
1  2  3  4  0   6   7   8   9   0  11  12  13  14   0  16  17  18  19   0  
6  7  8  9  0  11  12  13  14   0  16  17  18  19   0   0   0   0   0   0  


In [134]:
patches2 = get_centered_patches(3, test_image)
matprint(patches2)

0  0  0  0  0   0   0   1   2   3   0   5   6   7   8   0  10  11  12  13  
0  0  0  0  0   0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  
0  0  0  0  0   1   2   3   4   0   6   7   8   9   0  11  12  13  14   0  
0  0  1  2  3   0   5   6   7   8   0  10  11  12  13   0  15  16  17  18  
0  1  2  3  4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  
1  2  3  4  0   6   7   8   9   0  11  12  13  14   0  16  17  18  19   0  
0  5  6  7  8   0  10  11  12  13   0  15  16  17  18   0   0   0   0   0  
5  6  7  8  9  10  11  12  13  14  15  16  17  18  19   0   0   0   0   0  
6  7  8  9  0  11  12  13  14   0  16  17  18  19   0   0   0   0   0   0  


In [118]:
patches2.mean(axis=0)

array([-5.29819069e-08,  1.05963814e-07,  0.00000000e+00, -1.58945724e-07,
       -1.85436676e-07, -1.05963814e-07,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  2.64909545e-07,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.05963814e-07,
        0.00000000e+00, -3.17891448e-07,  3.17891448e-07, -5.29819069e-08],
      dtype=float32)

In [109]:
print(patches1[:,1])

[0. 0. 5. 0. 1. 6. 0. 2. 7.]


In [110]:
print(patches2[:,1])

[0. 0. 0. 0. 1. 2. 5. 6. 7.]


In [138]:
%%time

import cProfile

def compute_n_images():
    k = 3
    N = 600
    m = 64

    np.random.seed(0)

    all_patches = np.zeros((k ** 2, N * m ** 2))
    for i in range(N):
        #image_arr = Image.open()
        #gray = to_gray(image_arr)
        gray_im = np.random.rand(m, m)
        image_patches = get_centered_patches_profiled(k, gray_im) # k ** 2 x m ** 2
        #print(image_patches)
        # print(image_patches.shape, m**2)
        all_patches[:, i:i+m**2] = image_patches

    print(all_patches.shape)
    
    return all_patches

cProfile.run('compute_n_images()')

(9, 2457600)
         2494841 function calls (2493641 primitive calls) in 2.968 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      600    0.001    0.000    0.003    0.000 <__array_function__ internals>:2(around)
      600    0.001    0.000    0.030    0.000 <__array_function__ internals>:2(mean)
      600    0.001    0.000    0.047    0.000 <__array_function__ internals>:2(pad)
      600    0.001    0.000    0.005    0.000 <__array_function__ internals>:2(round_)
      600    1.626    0.003    2.922    0.005 <ipython-input-137-b873b8a1d142>:1(get_centered_patches_profiled)
        1    0.000    0.000    2.968    2.968 <string>:1(<module>)
        1    0.020    0.020    2.968    2.968 <timed exec>:3(compute_n_images)
     1200    0.000    0.000    0.003    0.000 _asarray.py:16(asarray)
      600    0.000    0.000    0.001    0.000 _asarray.py:88(asanyarray)
      600    0.011    0.000    0.025    0.000 _methods.py:134(_me

In [139]:
all_patches = compute_n_images()

(9, 2457600)


In [141]:
X = all_patches

In [ ]:
#Step4: Извлечение фильтров

In [149]:
from sklearn.decomposition import IncrementalPCA
n_filters = 8
pca = PCA(n_components=n_filters)

In [150]:
%time
pca.fit(X @ X.T)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs


PCA(copy=True, iterated_power='auto', n_components=8, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [152]:
# получаем таблицу - строка - номер фильтра, столбцы - flattend фильтр
pca.components_

array([[-0.32, -0.3 , -0.29, -0.31,  0.37,  0.35, -0.26,  0.38,  0.39],
       [-0.02, -0.02,  0.04,  0.02,  0.56,  0.45, -0.06, -0.42, -0.55],
       [-0.06,  0.04,  0.29, -0.13, -0.42,  0.52, -0.17, -0.5 ,  0.42],
       [ 0.1 , -0.17, -0.18, -0.17, -0.45,  0.5 ,  0.42,  0.34, -0.39],
       [ 0.15,  0.59,  0.17, -0.45, -0.06,  0.04, -0.49,  0.3 , -0.25],
       [ 0.7 ,  0.08, -0.61, -0.16,  0.06,  0.02, -0.03, -0.25,  0.2 ],
       [-0.26,  0.3 ,  0.01, -0.6 ,  0.19, -0.16,  0.61, -0.21,  0.12],
       [-0.45,  0.57, -0.53,  0.4 , -0.12,  0.16, -0.  , -0.04,  0.01]])

In [154]:
first_eigenvector = pca.components_[0].reshape(3,3)
first_eigenvector

array([[-0.32, -0.3 , -0.29],
       [-0.31,  0.37,  0.35],
       [-0.26,  0.38,  0.39]])

In [ ]:
class PCAFilter(filter_size, n_channels, im_size):
    def __init__(self,):
        self.pca = PCA(n_channels)
        self.filter_size = filter_size
        self.im_size = im_size
        self.n_channels = n_channels
        self.filters = None
    
    def get_stage1_filters(self, train_paths):
        N = len(train_paths)
        m = self.im_size
        k = self.filter_size
        # all patches of all images
        X = np.zeros((k ** 2, N * m ** 2))
        
        for i, path in enumerate(train_paths):
            image_array = load_image(path) ########
            image_array = image_array.resize((m, m)) ##########
            im_patches = self.get_centered_patches(image_array)
            X[:, i:i+m**2] = im_patches
        
        self.pca.fit(X @ X.T) # for incrementalPCA - partial_fit
        
        filters = np.zeros((n_channels, k, k))
        
        for i in range(n_channels):
            filters[i, :, :] = self.pca.components_[i].reshape(k, k)
        
        self.filters = filters
            
    def get_centered_patches(self, image) -> np.ndarray:
        padded_image = np.pad(image, (k // 2, k // 2), 'constant', constant_values=0) # можно не 0
        patches = []

        for i in range(image.shape[0]):
            for j in range(image.shape[1]):
                patch_matrix = padded_image[i:i+k, j:j+k]
                patches.append(patch_matrix)

        patches = np.array(patches, dtype=np.float32).T # k x k x m^2
        patches = patches.reshape(-1, X_im.shape[2]) # k^2 x m^2
        
        patch_mean = np.mean(patches, axis=0)
        patches -= patch_mean.reshape(-1, patch_mean.shape[0])
        return patches  

In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import torch
from torchvision import transforms, models

# from PIL import Image
# from colour_demosaicing import demosaicing_CFA_Bayer_Menon2007